In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
dl_oof_df = pd.read_csv('/kaggle/input/workshop-task-acl/deberta_v3_re_results/oof_df.csv')
mb_oof_df = pd.read_csv('/kaggle/input/workshop-task-acl/modern_bert_large_re_results/oof_df.csv')

dl_test_df = pd.read_csv('/kaggle/input/workshop-task-acl/deberta_v3_re_results/test_fold.csv')
mb_test_df = pd.read_csv('/kaggle/input/workshop-task-acl/modern_bert_large_re_results/test_fold.csv')

In [3]:
dl_test_proba_df = pd.read_csv('/kaggle/input/workshop-task-acl/deberta_v3_re_results/test_preds_proba.csv')
mb_test_proba_df = pd.read_csv('/kaggle/input/workshop-task-acl/modern_bert_large_re_results/test_preds_proba.csv')

In [4]:
w = (dl_test_proba_df*.3 + mb_test_proba_df*.7)

In [5]:
dl_test_df['preds_avg'] = np.argmax(w, axis = 1)
mb_test_df['preds_avg'] = np.argmax(w, axis = 1)

In [6]:
re_lab_dict = {'no_relation': 0,
 'Developer_of': 1,
 'URL_of': 2,
 'Citation_of': 3,
 'PlugIn_of': 4,
 'Version_of': 5,
 'Extension_of': 6,
 'Specification_of': 7,
 'Release_of': 8,
 'Abbreviation_of': 9,
 'License_of': 10,
 'AlternativeName_of': 11}

re_lab_inv_map = {v: k for k, v in re_lab_dict.items()}

In [7]:
dl_oof_df['relations'] = dl_oof_df['preds'].apply(lambda x: re_lab_inv_map[x])
dl_test_df['relations'] = dl_test_df['preds'].apply(lambda x: re_lab_inv_map[x])

In [8]:
dl_test_df['relations_avg'] = dl_test_df['preds_avg'].apply(lambda x: re_lab_inv_map[x])
mb_test_df['relations_avg'] = mb_test_df['preds_avg'].apply(lambda x: re_lab_inv_map[x])

In [12]:
def get_preds(df):
    # df['relations'] = df['preds'].apply(lambda x: re_lab_inv_map[x])
    df['relations'] = df['preds_avg'].apply(lambda x: re_lab_inv_map[x])
    df['entity_label_a'] = df['entity_token_a'].apply(lambda x: x.split(' [SEP] ')[0][2:])
    df['entity_label_b'] = df['entity_token_b'].apply(lambda x: x.split(' [SEP] ')[0][2:])
    df['relations_type'] = df['relations'].apply(lambda x: x.split('_')[0])
    df = df[df['entity_label_a']==df['relations_type']].reset_index(drop = True)
    df = df.drop(columns = ['entity_label_a', 'entity_label_b', 'relations_type', 'relation_text_2', 'train_text'])
    df.sort_values('document', inplace = True)
    df = df[['entity_token_1', 'entity_token_a', 'entity_token_b',
             'entity_token_2', 'document', 'relations', 'relations_avg']]
    return df

In [13]:
dl_test_df = get_preds(dl_test_df)
mb_test_df = get_preds(mb_test_df)

In [15]:
dl_test_df = dl_test_df.sort_values('document').reset_index(drop = True)

In [16]:
mb_test_df = mb_test_df.sort_values('document').reset_index(drop = True)

In [17]:
dl_test_df_1 = dl_test_df.drop(columns = ['relations_avg'])
mb_test_df_2 = mb_test_df.drop(columns = ['relations_avg'])

In [18]:
dl_test_df_1 = dl_test_df_1.drop(columns = ['entity_token_a', 'entity_token_b'])
mb_test_df_2 = mb_test_df_2.drop(columns = ['entity_token_a', 'entity_token_b'])

In [19]:
dl_test_df_1 = pd.concat([dl_test_df_1, mb_test_df_2]).drop_duplicates().reset_index(drop = True)

In [20]:
import pandas as pd

relation_dict = {}

for doc_id in range(220):
    if doc_id in dl_test_df_1['document'].unique():
        doc_rows = dl_test_df_1[dl_test_df_1['document'] == doc_id]
    
        formatted = "; ".join(
        f"{row['relations']}\t{row['entity_token_1']}\t{row['entity_token_2']}"
        for _, row in doc_rows.iterrows()
        )
    else:
        formatted = ''
    relation_dict[doc_id] = formatted

with open("predictions.relations.txt", "w") as f:
    for doc, text in relation_dict.items():
        f.write(f"{text}\n")